In [1]:
import glob
import os
from tqdm import tqdm
import rasterio
import numpy as np
import geopandas as gpd
import time
import shutil
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import fiona

In [2]:
import areacal
import importlib
importlib.reload(areacal)
from areacal import countraster
from areacal import grwlext
from areacal import shptoraster

<module 'areacal' from 'L:\\code\\areacal.py'>

In [21]:
tif_list = glob.glob(r'P:\tif\0\*.tif') 
tif_list[310:320]

['P:\\tif\\0\\ndwi1_int8_world_351.tif',
 'P:\\tif\\0\\ndwi1_int8_world_352.tif',
 'P:\\tif\\0\\ndwi1_int8_world_353.tif',
 'P:\\tif\\0\\ndwi1_int8_world_354.tif',
 'P:\\tif\\0\\ndwi1_int8_world_355.tif',
 'P:\\tif\\0\\ndwi1_int8_world_357.tif',
 'P:\\tif\\0\\ndwi1_int8_world_358.tif',
 'P:\\tif\\0\\ndwi1_int8_world_359.tif',
 'P:\\tif\\0\\ndwi1_int8_world_360.tif',
 'P:\\tif\\0\\ndwi1_int8_world_361.tif']

In [16]:
num = 17000
tif_list = glob.glob(fr'D:\zwk_data\p_tif\{num}\*.tif')  # 要处理的tif

dgrwldir = cdir(fr'D:\zwk_data\p_tif\{num}\dgrwl')
dreserdir = cdir(fr"D:\zwk_data\p_tif\{num}\dreser")
temptxt = os.path.join(dreserdir,'rivercount.txt')
skeldir = cdir(fr'D:\zwk_data\p_tif\{num}\skel')
disdir = cdir(fr"D:\zwk_data\p_tif\{num}\dis")

#grwltif = r'P:\tif\0\temp'

cpdir = cdir(fr"D:\zwk_data\p_tif\{num}\comp")
cdir(os.path.join(cpdir,'big'))
cdir(os.path.join(cpdir,'river'))

dirvec = r"D:\zwk_data\p_tif\GRWL_vector_V01.01"
#dir = r'P:\tif\0'
txt = r'P:\tif\0\log.txt'


'D:\\zwk_data\\p_tif\\17000\\comp\\big'

'D:\\zwk_data\\p_tif\\17000\\comp\\river'

In [21]:
num = 17000
tif_list = glob.glob(fr'D:\zwk_data\p_tif\{num}\*.tif')  # 要处理的tif

dgrwldir = cdir(fr'D:\zwk_data\p_tif\{num}\dgrwl_3')
dreserdir = cdir(fr"D:\zwk_data\p_tif\{num}\dreser_3")
temptxt = os.path.join(dreserdir,'rivercount_3.txt')
skeldir = cdir(fr'D:\zwk_data\p_tif\{num}\skel_3')
disdir = cdir(fr"D:\zwk_data\p_tif\{num}\dis_3")

#grwltif = r'P:\tif\0\temp'

cpdir = cdir(fr"D:\zwk_data\p_tif\{num}\comp")
cdir(os.path.join(cpdir,'big'))
cdir(os.path.join(cpdir,'river'))

dirvec = r"D:\zwk_data\p_tif\GRWL_vector_V01.01"
#dir = r'P:\tif\0'
txt = r'P:\tif\0\log.txt'


'D:\\zwk_data\\p_tif\\17000\\comp\\big'

'D:\\zwk_data\\p_tif\\17000\\comp\\river'

In [22]:

for intif in tqdm(tif_list):
    print(intif)
    rasterbound = extofraster(intif)
    x = (rasterbound[0]+rasterbound[2])/2             
    y = (rasterbound[1]+rasterbound[3])/2
    
    intifnum = countraster(intif, num = 1)
    drnum = countraster(intif, num = 1)
    s_sg_rivernum = 0
    s_g_rivernum = 0
    grwlvec = grwlext(rasterbound, dirvec, txt, extt = '.shp')
    #合并shp
    smallriver = gpd.GeoDataFrame()
    time.ctime()
    for shpname in grwlvec:
        gdf = gpd.read_file(shpname)
        if len(gdf) != 0:
            smallriver = smallriver.append(gdf, ignore_index=True)
    
    if len(smallriver) != 0:
        allgr = riverbuffer(smallriver,intif)
        #去除grwl所有河流来计算中心线
    if len(allgr) !=0:
        dgrwlpath = os.path.join(dgrwldir, "dgrwl_"+os.path.split(intif)[1])
        rasterminusdf(intif,allgr,dgrwlpath, invert = True, geom = 1)
    else:
        dgrwlpath = intif
        #drnum = countraster(dreserpath, num = 1)
    skelpath = os.path.join(skeldir, "skel_"+os.path.split(intif)[1])
    dispath = os.path.join(disdir, "dis_"+os.path.split(intif)[1])
    skeldis(dgrwlpath,skelpath,dispath)
    #得到 all grwl的buffer 和 reservior的buffer
    if len(allgr) !=0:
        grwlriver = allgr[allgr['lakeFlag'] != 1]
        grwllake = allgr[allgr['lakeFlag'] == 1]
    
    # intif去除reservior 重新计算新的面积

        if len(grwllake) !=0:
            dreserpath = os.path.join(dreserdir, "dreser_"+os.path.split(intif)[1])
            rasterminusdf(intif,grwllake,dreserpath, invert = True, geom = 1)
            drnum = countraster(dreserpath, num = 1)

            #big 和 all river 的去水库
            cpriver1 = os.path.join(cpdir,'big',   os.path.split(intif)[1])
            cpriver2 = os.path.join(cpdir,'river', os.path.split(intif)[1])    
            if os.path.exists(cpriver1):
                count1 = countraster(cpriver1, num = 2)
                s_sg_rivernum = count1 + drnum
            if os.path.exists(cpriver2):
                count2 = countraster(cpriver2, num = 4)
                s_g_rivernum = count2 + drnum
    
    #s_sg_rivernum = 0
    #s_g_rivernum = 0
    intxt =  f"{intif}\t{intifnum}\t{drnum}\t{s_sg_rivernum}\t{s_g_rivernum}\t{x}\t{y}\n"
    with open(temptxt, 'a') as txt_file:
        txt_file.write(intxt)
intxt =  f"{'intif'}\t{'intifnum'}\t{'drnum'}\t{'s_sg_rivernum'}\t{'s_g_rivernum'}\t{x}\t{y}\n"        
with open(temptxt, 'a') as txt_file:
    txt_file.write(intxt)        
copytxt = os.path.join(dreserdir, 'area.txt')
shutil.copy(temptxt,copytxt)
time.ctime()


  0%|                                                                                          | 0/574 [00:00<?, ?it/s]

D:\zwk_data\p_tif\17000\ndwi1_int8_world_17003.tif
['D:\\zwk_data\\p_tif\\GRWL_vector_V01.01\\SC55.shp']


'Fri Dec 29 11:16:40 2023'

108

  0%|▏                                                                               | 1/574 [00:41<6:38:02, 41.68s/it]

...

107

100%|█████████████████████████████████████████████████████████████████████████████| 574/574 [13:09:37<00:00, 82.54s/it]


84

'D:\\zwk_data\\p_tif\\17000\\dreser_3\\area.txt'

'Sat Dec 30 00:26:16 2023'

In [18]:
def cdir(name):
    os.makedirs(name, exist_ok=True)
    return name

def mask1(shp,intif, outtif, all_touch = False,crop = False, pad = False, mod = 1):
    with fiona.open(shp,'r') as shapefile:
        shapes = [feature['geometry'] for feature in shapefile]
        #print(shapefile.crs)
    if shapes:
        with rasterio.open(intif, 'r') as dst:
            out,_ = rasterio.mask.mask(dst, shapes, all_touched = all_touch ,crop = crop , pad = pad)
            pro = dst.profile
            #print(dst.crs)
        if mod == 1:
            with rasterio.open(outtif,'w',**pro) as dst:
                dst.write(out)
        else:
             return out
  
def rasterminus(rasterp1, rasterp2):
    with rasterio.open(rasterp1) as src1, rasterio.open(rasterp2) as src2:
        # 读取栅格数据
        raster1 = src1.read(1)
        raster2 = src2.read(1)

        # 确保两个栅格的大小相同
        if raster1.shape == raster2.shape:
            # 相减操作
            result = raster1 - raster2
            
            return result
        else:
            print('大小不一样')

def readrasornot(rasterp1, pa1):
    if pa1 == 1:
        with rasterio.open(rasterp1) as src1:
            raster1 = src1.read(1)
    else:
        raster1 = rasterp1
    return raster1

def rasteradd(rasterp1, rasterp2, pa1 = 1, pa2 = 1):
    raster1 = readrasornot(rasterp1, pa1)
    raster2 = readrasornot(rasterp2, pa2)

    # 确保两个栅格的大小相同
    if raster1.shape == raster2.shape:
        # 相减操作
        result = raster1 + raster2    
        return result
    else:
        print('大小不一样')
        
def riverbuffer(gdf, intif):
    
    #gdf = gdf[gdf['width_m'] < 90]
    gdf = gdf.to_crs("EPSG:3857")
    gdf['geometry'] = gdf.buffer(gdf['width_m']*3)
    gdf = gdf.to_crs("EPSG:4326")
    if len(gdf) == len(gdf[gdf.is_valid]):
        return gdf 
    else:
        print('invalid')
        with open(r"P:\tif\invalidgrwlbuffer.txt",'a') as txt:
            txt.write(intif+'\n')
        return gpd.GeoDataFrame()


def countraster(rasterpath, num = 1):
    with rasterio.open(rasterpath) as src:
        # 读取栅格数据
        raster_data = src.read(1)
        cnum = np.sum(raster_data == num)
    return cnum

def extofraster(tifpath):
    with rasterio.open(tifpath) as src:
        bounds = src.bounds
    ext = [bounds.left,bounds.bottom,bounds.right,bounds.top]
    return ext

from skimage import morphology
def skeldis(intif,skelpath,dispath):
    with rasterio.open(intif) as src:
        data = src.read(1)
        pro = src.profile
        dtype = src.dtypes[0]
    skel, distance =morphology.medial_axis(data, return_distance=True)
    skel = skel.astype(dtype)
    distance = np.round(distance).astype(dtype)
    #skel_dis = skel*distance
    with rasterio.open(skelpath,'w',**pro) as dst:
        dst.write(skel,1)
    with rasterio.open(dispath,'w',**pro) as dst:
        dst.write(distance,1)
    #return skel, distance

def rasterminusdf(existing_raster_path,shapefile_path,output_raster_path, invert = True, geom = 0):
    # Read the existing raster to use as a template
    with rasterio.open(existing_raster_path) as src:
        # Get raster properties
        crs = src.crs
        transform = src.transform
        width = src.width
        height = src.height
        dtype = src.dtypes[0]  # Assuming a single band raster
        data = src.read(1)
    # Read the shapefile
    if geom == 0:
        gdf = gpd.read_file(shapefile_path)
    else:
        gdf = shapefile_path
    if len(gdf) !=0:
        gdf = gdf.to_crs(crs)
        # Create an empty raster with the same properties as the existing raster
        with rasterio.open(
            output_raster_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=1,  # Number of bands
            dtype=dtype,
            crs=crs,
            transform=transform,
            compress = 'LZW'
        ) as dst:
            # Rasterize the geometry onto the raster
            mask = rasterio.features.geometry_mask(
                gdf.geometry,
                transform=transform,
                invert= invert,
                #all_touched = True,
                out_shape=(height, width)
            )
            #mask = data - mask
            mask = np.where((mask == 1) & (data == 1), 0, data)
            # Write the rasterized data to the output raster
            dst.write(mask.astype(dtype), 1)
    else:
        if geom ==0:
            print(shapefile_path)
        else:
            print(existing_raster_path)
        print('no data')